### Solution 1 class 

In [2]:
import ray
from ray import serve
import requests  # for making web requests

@serve.deployment
class FruitStand:
    def __init__(self, fruit):
        self.fruit = fruit
        self.prices  = {"mango": 2.56, "pear": 1.75, "apples": 1.35}
    
    def compute_price(self, data):
        return self.prices[self.fruit] * data

    def __call__(self, starlette_request):
        data = float(starlette_request.query_params['data'])
        price = self.compute_price(data)
        return {"prices": price}

In [3]:
simple_model = FruitStand.bind("mango")
serve.run(simple_model)

2022-08-09 18:59:25,041	INFO worker.py:1481 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265.
(ServeController pid=88014) INFO 2022-08-09 18:59:26,212 controller 88014 http_state.py:123 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-9d080018d970d4e5d5f28fd366301c0a180d47987e411871e96ecac9' on node '9d080018d970d4e5d5f28fd366301c0a180d47987e411871e96ecac9' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=88016) INFO:     Started server process [88016]
(ServeController pid=88014) INFO 2022-08-09 18:59:26,830 controller 88014 deployment_state.py:1232 - Adding 1 replicas to deployment 'FruitStand'.
You are retrieving a sync handle inside an asyncio loop. Try getting Deployment.get_handle(.., sync=False) to get better performance. Learn more at https://docs.ray.io/en/latest/serve/handle-guide.html#sync-and-async-handles


RayServeSyncHandle(deployment='FruitStand')

In [6]:
url = f"http://127.0.0.1:8000/FruitStand"
for i in range(5):
    print(f" {i+1} mango prices  : {requests.get(url, params={'data': i+1}).text}")

 1 mango prices  : {"prices": 2.56}
 2 mango prices  : {"prices": 5.12}
 3 mango prices  : {"prices": 7.68}
 4 mango prices  : {"prices": 10.24}
 5 mango prices  : {"prices": 12.8}


(HTTPProxyActor pid=88016) INFO 2022-08-09 19:01:40,712 http_proxy 127.0.0.1 http_proxy.py:315 - GET / 200 5.1ms
(HTTPProxyActor pid=88016) INFO 2022-08-09 19:01:40,718 http_proxy 127.0.0.1 http_proxy.py:315 - GET / 200 1.7ms
(HTTPProxyActor pid=88016) INFO 2022-08-09 19:01:40,722 http_proxy 127.0.0.1 http_proxy.py:315 - GET / 200 1.5ms
(HTTPProxyActor pid=88016) INFO 2022-08-09 19:01:40,726 http_proxy 127.0.0.1 http_proxy.py:315 - GET / 200 1.8ms
(HTTPProxyActor pid=88016) INFO 2022-08-09 19:01:40,729 http_proxy 127.0.0.1 http_proxy.py:315 - GET / 200 1.6ms
(ServeReplica:FruitStand pid=88019) INFO 2022-08-09 19:01:40,710 FruitStand FruitStand#qFQzCu replica.py:482 - HANDLE __call__ OK 0.5ms
(ServeReplica:FruitStand pid=88019) INFO 2022-08-09 19:01:40,718 FruitStand FruitStand#qFQzCu replica.py:482 - HANDLE __call__ OK 0.1ms
(ServeReplica:FruitStand pid=88019) INFO 2022-08-09 19:01:40,721 FruitStand FruitStand#qFQzCu replica.py:482 - HANDLE __call__ OK 0.1ms
(ServeReplica:FruitStand pi

In [8]:
serve.shutdown()

(ServeController pid=88014) INFO 2022-08-09 19:02:20,983 controller 88014 deployment_state.py:1257 - Removing 1 replicas from deployment 'FruitStand'.
